# writing command line applications

## using *plumbum*

Command line applications are a generalization of the idea of script: the execution of the program is conditioned on some parameters provided by the user.

This is typically done using **options**, **parameters** and **flags**, and are the typical interface used by bash programs.

In this lesson we will (quickly) overview how to implement one of these program, leveraging a 3rd party library called **plumbum**

Everything we are doing today with **plumbum** can be done in a reasonable way in basic python, but plumbum will provide us a more simple and consistent way of doing it.

In [1]:
import plumbum
# this script uses plumbum version 1.6.7
print(plumbum.__version__)

(1, 6, 7)


## Why applications?

functionalities implemented as CLI applications share a lot of the advantages with libraries

1. Commands are repeatable
2. Commands can be shared
3. Commands ca be automated (and this scales well)
4. Commands should be readable
5. Commands can be composed together

## Preface - utilities

Plumbum provides us some simple utilities to quickly improve the experience of the user when using the application we are writing

### colored text output

```python
from plumbum import colors
# this will not work in jupyter notebooks
print(colors.green & colors.bold | "This is green and bold.")
```

### image visualization in the terminal

```python
from PIL import Image as imreader
from plumbum.cli.image import Image as imdisplay
im = imreader.open("fractal_wrongness.png")
# does not work with jupyter
imdisplay().show_pil(im)
```

### progress bars

In [31]:
# progress bars
from plumbum.cli.terminal import Progress
import time

for i in Progress.range(10):
    time.sleep(0.2)

### configurations files

In [32]:
%%file .myapp_rc
[DEFAULT]
three=3

[OTHER]
a=3

Overwriting .myapp_rc


In [33]:
from plumbum import cli

with cli.ConfigINI('.myapp_rc') as conf:
    one = conf['three']
    two = conf.get('one', default='2')
    three = conf.get('OTHER.a')
    
    # changing the configuration file
    conf['OTHER.b'] = 4
print(one, two, three)

3 2 3


In [34]:
!cat .myapp_rc

[DEFAULT]
three = 3
one = 2

[OTHER]
a = 3
b = 4



### reading user input

In [36]:
import plumbum.cli.terminal as terminal
# see also ask and prompt
terminal.choose("favorite color?", ['red', 'green', 'blue'], default='blue')

favorite color?
(1) red
(2) green
(3) blue
Choice [3]: 

'blue'

### the local computer

In [43]:
from plumbum import local
ls = local["ls"]
print(ls().splitlines()[:4])

['Code dump.ipynb', 'Conda_Environment_instructions.txt', 'directory_structure.png', 'directory_structure.svg']


the `local.cmd` dynamically loads programs from the environment.

There is actually no such package, everything is created on the fly!

In [44]:
from plumbum.cmd import grep, cat

Programs can be combined in pipes in the same way as one could do in the bash shell.

these pipes, in the same way as the individual programs, are not run until called

In [49]:
ls|grep['.png']

Pipeline(LocalCommand(/bin/ls), BoundCommand(LocalCommand(/bin/grep), ['.png']))

In [52]:
pipe = ls|grep['.png']
print(pipe())

directory_structure.png
file_inode_permissions.png
fractal_wrongness.png
OS_structure.png
__temp_ipython__.png
users_and_groups.png



### background and foreground execution
```python
from plumbum import FG, BG
ls["-l"] & FG
```

### input-output redirection
```python
import sys
((grep["world"] < sys.stdin) > "tmp.txt")()
cat("tmp.txt")
```

In [56]:
pipe = (cat << "hello world\nfoo\nbar\spam" | grep["oo"])
pipe()

'foo\n'

In [170]:
# temporarely change the working directory
with local.cwd(local.cwd / "myapp"):
    print(local.cwd)

/home/enrico/didattica/corso_programmazione_1819/programmingCourseDIFA/myapp


### remote connection to other machines

In [ ]:
from plumbum import SshMachine
rem = SshMachine("hostname", user = "john", keyfile = "/path/to/idrsa")
rem.close()

In [ ]:
# connect to a remote machine 
with SshMachine("hostname", user = "john", keyfile = "/path/to/idrsa") as rem:
    pass

In [ ]:
# temporarely change the working directory in the remote server
with rem.cwd(rem.cwd / "Desktop"):
    print(rem.cwd)

In [ ]:
# port tunneling
with rem.tunnel(6666, 8888):
    pass

In [ ]:
# local grep of a remote ls
from plumbum.cmd import grep
r_ls = rem["ls"]
pipe = r_ls | grep["b"]
pipe()

# Command line programs

In python any script can be at the same time a library and a command line program.

All the code in the general section of the script is executed everytime the script is imported as a library, but one can set a part of the code to be run only when the script is called as a command line program.

this is achieved by testing the `__name__` parameter of the program (a special variable that the interpreter set at the start), that when executed as a program is set to the string `"__main__"`.

In [183]:
%%file my_first_app.py

def mysum(a, b):
    return a+b

print("this will be executed everytime")

if __name__=='__main__':
    print("this will be executed only on the command line")

Writing my_first_app.py


In [185]:
# this execute a single python command and quit
!python3 -c "import my_first_app"

this will be executed everytime


In [186]:
!python3 ./my_first_app.py

this will be executed everytime
this will be executed only on the command line


This method could be used directly to implement very simple applications, by reading the arguments using `sys.argv`, and then parsing them by hand, but it's not very comfortable

In [189]:
%%file my_first_app.py
if __name__=='__main__':
    import sys
    print(sys.argv)

Overwriting my_first_app.py


In [190]:
!python3 ./my_first_app.py arg1 arg2

['./my_first_app.py', 'arg1', 'arg2']


Python comes with a module that provide automatic arguments and options parsing, and works reasonably well, called `argparse`, that can be used in situations where external libraries can't be installed.

the official tutorial can be found on the [python documentation](https://docs.python.org/dev/howto/argparse.html)

It works, but it creates a code that is usually hard to read and manage when the options get more complicated.

it is also missing all the support classes and functions that plumbum provides and that make easier to create high quality CLI

In [193]:
%%file my_first_app.py
if __name__=='__main__':
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument("echo", help="echo the string you use here")
    
    args = parser.parse_args()
    
    print("the received string is: {}".format(args.echo))

Overwriting my_first_app.py


In [195]:
!python3 ./my_first_app.py --help

usage: my_first_app.py [-h] echo

positional arguments:
  echo        echo the string you use here

optional arguments:
  -h, --help  show this help message and exit


In [194]:
!python3 ./my_first_app.py to_be_said

the received string is: to_be_said


# Command line classes

command line applications can be imagined as structured in a similar way to a function:

There are a certain number of ordinal arguments, and a certain number of named options.

The arguments are the main focus of the program, while the option determines exactly how the execution is performed.

The design approach of plumbum is that a program is a class.

Options and flags connect to methods that configure the **self** attributes

Parameters are given to a **main** function that is the method that performs the program execution.

The **main** function have access to the results of the configuration as the configured **self** instance.

In [176]:
%%file my_app.py
from plumbum import cli

class MyApp(cli.Application):
    """returns the square of a number
    """
    PROGNAME = "MyGloriousApp"
    VERSION = "0.1"
    
    
    def main(self, value: float):
        #value = float(value)
        print("result: {}".format(value**2))

if __name__ == "__main__":
    MyApp()

Overwriting my_app.py


In [177]:
!python3 my_app.py --help

MyGloriousApp 0.1

returns the square of a number

Usage:
    MyGloriousApp [SWITCHES] value

Meta-switches:
    -h, --help         Prints this help message and quits
    --help-all         Prints help messages of all sub-commands and quits
    -v, --version      Prints the program's version and quits



In [180]:
# type decoration is used to enforce arguments type and convert them.
!python3 my_app.py 3

result: 9.0


In [178]:
# type decoration is used to enforce arguments type and convert them.
!python3 my_app.py a

Error: Argument of value expected to be <class 'float'>, not 'a':
    ValueError("could not convert string to float: 'a'",)
------
MyGloriousApp 0.1

returns the square of a number

Usage:
    MyGloriousApp [SWITCHES] value

Meta-switches:
    -h, --help         Prints this help message and quits
    --help-all         Prints help messages of all sub-commands and quits
    -v, --version      Prints the program's version and quits



In [65]:
%%file my_app.py
from plumbum import cli

class MyApp(cli.Application):
    PROGNAME = "MyGloriousApp"
    VERSION = "0.1"
    
    verbose = cli.Flag(["v", "verbose"], help = "If given, I will be very talkative")

    def main(self, filename):
        print("I will now read {0}".format(filename))
        if self.verbose:
            print("Yadda " * 200)

if __name__ == "__main__":
    MyApp()

Overwriting my_app.py


Flags and switches can be:

* compulsory
* dependent on each other
* mutually exclusive
* repeatable
* gouped together in the help description
* bound to have specific values

In [66]:
!python3 my_app.py --help

MyGloriousApp 0.1

Usage:
    MyGloriousApp [SWITCHES] filename

Meta-switches:
    -h, --help         Prints this help message and quits
    --help-all         Prints help messages of all sub-commands and quits
    --version          Prints the program's version and quits

Switches:
    -v, --verbose      If given, I will be very talkative



In [67]:
!python3 my_app.py foo

I will now read foo


In [69]:
!python3 my_app.py foo --verbose

I will now read foo
Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Ya

In [68]:
!python3 my_app.py foo -v

I will now read foo
Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Yadda Ya

In [72]:
%%file my_app.py
from plumbum import cli

class MyApp(cli.Application):
    PROGNAME = "MyGloriousApp"
    VERSION = "0.1"
    _log_file = None
    
    @cli.switch("--log-to-file", str)
    def log_to_file(self, filename):
        """Sets the file into which logs will be emitted"""
        self._log_file = filename

    def main(self, filename):
        print("I will now read {0}".format(filename))
        if self._log_file is not None:
            with open(self._log_file, 'w') as outfile:
                print("I will now read {0}".format(filename), file=outfile)

if __name__ == "__main__":
    MyApp()

Overwriting my_app.py


In [78]:
!rm -f temp_log.txt

In [79]:
!python3 my_app.py foo --log-to-file temp_log.txt

I will now read foo


In [80]:
!cat temp_log.txt

I will now read foo


### sub commands

sub-commands are a way to group together several related program under a single umbrella.

a classical example is **git**.

In [127]:
%%file my_app.py
from plumbum import cli

class MyApp(cli.Application):
    """returns the square of a number
    """
    PROGNAME = "MyGloriousApp"
    VERSION = "0.1"
    
    def main(self):
        if not self.nested_command:           # will be ``None`` if no sub-command follows
            print("No command given")
            return 1   # error exit code

@MyApp.subcommand("pow")
class MyAppPow(cli.Application):
    "calculate the square power instead"
    def main(self, value, power=2):
        value = float(value)
        power = float(power)
        print("result: {}".format(value**power))    
    
@MyApp.subcommand("sqrt")
class MyAppSqrt(cli.Application):
    "calculate the square root instead"
    def main(self, value):
        value = float(value)
        print("result: {}".format(value**0.5))
        
if __name__ == "__main__":
    MyApp()

Overwriting my_app.py


In [128]:
!python3 my_app.py --help

MyGloriousApp 0.1

returns the square of a number

Usage:
    MyGloriousApp [SWITCHES] [SUBCOMMAND [SWITCHES]] 

Meta-switches:
    -h, --help         Prints this help message and quits
    --help-all         Prints help messages of all sub-commands and quits
    -v, --version      Prints the program's version and quits

Sub-commands:
    pow                calculate the square power instead; see 'MyGloriousApp
                       pow --help' for more info
    sqrt               calculate the square root instead; see 'MyGloriousApp
                       sqrt --help' for more info


In [129]:
!python3 my_app.py sqrt --help

MyGloriousApp sqrt 0.1

calculate the square root instead

Usage:
    MyGloriousApp sqrt [SWITCHES] value

Meta-switches:
    -h, --help         Prints this help message and quits
    --help-all         Prints help messages of all sub-commands and quits
    -v, --version      Prints the program's version and quits



In [133]:
!python3 my_app.py pow --help

MyGloriousApp pow 0.1

calculate the square power instead

Usage:
    MyGloriousApp pow [SWITCHES] value [power=2]

Meta-switches:
    -h, --help         Prints this help message and quits
    --help-all         Prints help messages of all sub-commands and quits
    -v, --version      Prints the program's version and quits



In [130]:
!python3 my_app.py sqrt 9

result: 3.0


In [131]:
!python3 my_app.py pow 3

result: 9.0


In [132]:
!python3 my_app.py pow 3 0.5

result: 1.7320508075688772


## system wide installation

to perform a system wide installation we will use the `setuptools` package, that allows us to interface our program with the pip installation mechanism.

### preparing the field

The first step is to create a proper folder structure for our program.
In this case I'm creating a CLI program called **myapp**.

* **myapp** (project folder)
    * `setupy.py`
    * **myapp** (program folder)
        * `__init__.py`
        * `__main__.py`
        

the outside folder is the project folder, that will contain everything concerning my app: the app itself, its documentation, the installation procedure, and so on.

Then there is an inside folder, usually with the same name of the app, that will contain the proper files to be executed.

Inside this folder there should be a file `__init__.py`.
This file can be empty, but is used by python to recognize that this directory is a module that can be imported or executed.

My application is going to be in the `__main__.py` file. No special reason for the name, just more consistant

In [154]:
!touch ./myapp/myapp/__init__.py

In [153]:
%%file ./myapp/myapp/__main__.py
from plumbum import cli

class MyApp(cli.Application):
    """returns the square of a number
    """
    PROGNAME = "MyGloriousApp"
    VERSION = "0.1"
    
    def main(self):
        if not self.nested_command:           # will be ``None`` if no sub-command follows
            print("No command given")
            return 1   # error exit code

@MyApp.subcommand("pow")
class MyAppPow(cli.Application):
    "calculate the square power instead"
    def main(self, value, power=2):
        value = float(value)
        power = float(power)
        print("result: {}".format(value**power))    
    
@MyApp.subcommand("sqrt")
class MyAppSqrt(cli.Application):
    "calculate the square root instead"
    def main(self, value):
        value = float(value)
        print("result: {}".format(value**0.5))
        
if __name__ == "__main__":
    MyApp.run()

Overwriting ./myapp/myapp/__main__.py


The file necessary for setuptools to work properly is the `setup.py`, where we will use the `setup` function of the `setuptools` module.

This function configure many things for us, but the most important thing is the `entry_point` parameter.
This creates the link to the scripts that we will actually execute.
Inside this parameter we specify that two entry points are `console_scripts` that should execute the script and the function provided.

In [160]:
%%file ./myapp/setup.py
from setuptools import setup

setup(
    name = 'myapp',
    version = '0.1.0',
    packages = ['myapp'],
    install_requires=[ 'plumbum', ],
    entry_points = {
        'console_scripts': [
            'myapp = myapp.__main__:MyApp',
            'myappsqrt = myapp.__main__:MyAppSqrt',
        ]
    })

Overwriting ./myapp/setup.py


In this case we decided to let the user use the standard program (choosing the sub command to use) or to provide a shortcut to the subcommand itself.

In this case the plumbum structure comes very useful: we can call directly the subcommand class to execute our subcommand.

the name we put before the `=` sign is going to be the command name that we will use to call the program from the command line

Last special mention goes to the `install_requires`: this list allows to specify all the dependencies of the package.
If they are not installed, they will automatically downloaded and installed by pip.
In our case the only dependency that we have is for the `plumbum` package

### the installation step

from the folder containing the project folder, we can call the pip installation program

by installing it with the `--editable` (or `-e` for short) the program will not be copied in the system, but rather linked in it.
This means that if you edit the program, the modifications will appear live in the systems without need to reinstall

In [162]:
!pip3 install --editable myapp

Obtaining file:///home/enrico/didattica/corso_programmazione_1819/programmingCourseDIFA/myapp
  Running setup.py develop for myapp


In [157]:
!myapp --help

MyGloriousApp 0.1

returns the square of a number

Usage:
    MyGloriousApp [SWITCHES] [SUBCOMMAND [SWITCHES]] 

Meta-switches:
    -h, --help         Prints this help message and quits
    --help-all         Prints help messages of all sub-commands and quits
    -v, --version      Prints the program's version and quits

Sub-commands:
    pow                calculate the square power instead; see 'MyGloriousApp
                       pow --help' for more info
    sqrt               calculate the square root instead; see 'MyGloriousApp
                       sqrt --help' for more info


We can call the base program or the specific subcommand, as we specified

In [158]:
!myapp sqrt 9 

result: 3.0


In [159]:
!myappsqrt 9

result: 3.0


Lastly, we can uninstall the program from any location

In [161]:
!pip3 uninstall -y myapp

Uninstalling myapp-0.1.0:
  Successfully uninstalled myapp-0.1.0


If one needs to shop non-code files (such as images, configuration files, etc...)
check:

https://python-packaging.readthedocs.io/en/latest/non-code-files.html

given that now **myapp** is a system program, I can load it using `plumbum` as any other application!

In [168]:
myapp_hook = local['myapp']
sqrt_hook = myapp_hook['sqrt']
print(sqrt_hook(9))

result: 3.0



## CLI application good practices

### Errors management


#### clean up after yourself, especially on errors
If something goes wrong, try to avoid leaving the system in an invalid state, and clean unnecessary files

#### allow if possible to recover partial results
This is especially important in long executions: take a hint from the snakemake approach, and try to have partial results stored as temporary files to avoid unnecessary repetition when possible

#### try to be robust to bad data and configuration
it is better to do less things, but making sure that those things are well covered and the program will not trow a fit for some input data in an awkward format.

Utilities should handle arbitrarily long lines, huge files, and so on.
It is perhaps tolerable for a utility to fail on a data set larger than it can hold in memory, but some utilities don't do this.
for instance, sort, by using temporary files, can generally sort data sets much larger than it can hold in memory. 

#### Exit with different error codes
this is the values that is returned by the **main**
0 means success, any other number means failure
the best solution is to code each error to a specific return value (and document it)

### User interaction


#### give feedback for task longer than a second or two (such as progress bars)
Users might worry that something has gone wrong and just kill the process if they have to wait without any prompt that the program is doing something.
Even better, try to use a progress bar with a time estimate (plumbum has you covered)

#### let the user control the level of verbosity of the output
include a `--verbose`, a `--terse` and a `--silent` option to allow the user to control the level of feedback they wants

#### Colour code your output

Ideally, your script should:
* output white/default (it’s the foreground process), 
* child processes should output grey (usually not needed unless things screw up), 
* success should be denoted with green, 
* failure, red, and 
* warnings in yellow.

#### Have both short ie one letter (Eg `-h`) and long forms (eg `--help`) of the command's switches
long format helps readability and long term maintanability, and are good for scripting and manuals

short forms are useful for manual input, especially of commonly used options (don't waste the user time)

#### Ask for confirmation for irreversible action
such as deleting data, changing informations without backup, etc...

there should be an option to make this automatic, but the user should be protected from involuntary changes to the system.

Even better, while asking for confirmation, also suggest which is the option to turn it off

### Documentation

#### consider adding a `help` sub command
alongside the `--help` flag, consider including an help command that contains information about various topics

#### provide some use cases in the documentation
Especially if the case uses are not trivial, with many interlocked features interacting with each other.

And, is possible, put them nearby the start of the documentation, rather than at the end.

#### In case of mistakes try suggesting common correction techniques
Git again does this perfectly: when something goes wrong, it also suggest reliable methods to correct the bad situation, without covering this information in "output noise"

#### make common commands memorable
Don't use weird names, inside jokes, abstract references, words referring to implementation details (unless relevant).

keep it simple and easy to remember.

### Configuration


#### Work independently of current working directory
use only absolute paths (/path/to/something) and paths relative to the script (demonstrated below)

#### consider using configuration files
This allows a greater deal of automation and configurability, reducing the load on the user to write all the options by hand.

on the other hand, a configuration file should never be a necessity, and everything should be used as an option

#### all the options that can have a default value, should have one
Ideally, the program should be executable without specifying any option.

compulsory options are probably candidates for sub-commands

### Interface

#### try to be consistent with other GNU tools : 
For example see a list of options from other GNU tools and if possible try to align to them

http://www.catb.org/esr/writings/taoup/html/ch10s05.html

https://www.gnu.org/prep/standards/standards.html

#### play well with pipes and allow input and output to files
To play well with pipes it should read and write to stdout by default, but the user should be free to redirect to file if necessary

#### If a command has a side effect provide a dry-run/whatif/no_post option.
don't let the user jump blindly without checking what would happen.

This also allows the user to find possible mistakes faster and with less risks

Snakemake perfors this very well.

#### everything should be doable in a script, without human interaction
To allow the program to be fully used in script and automated, it should always be possible to execute it without any input from the user.

When possible, this should be the preferred behavior

#### try to make it composable
programs that play nicely with other can be used to implement even more complicated pipelines and programs.

#### conside adding switches to both turn on and off behaviors
If a switch is in the long-form (for example --foo), which turns “on” some behavior (that is off by default), there should also be another switch that turns “off” the behavior.

This allow the user to be more explicit and change-proof the script for future change of default behavior

#### consider using a long informative name for you app

the GNU convention is to use 2 or 3 letters names, but if one can avoid it, the deafult program name should be something either explicit or memorable.

The user has always the option to alias it to something shorter.

#### if no output is necessary, avoid output unless required
that's what the `--verbose` option is for.

if the program performs an action and doesn't have to return a result in case of success (such as moving a file, changing directory, etc...), the user should see an output only in case of errors or if the verbose option is selected.

This makes the program more script friendly

# Exercise

Write a personal knowledge database app.

this app should:
* add new notes
* show notes
* search notes based on words contained in them

https://codeburst.io/13-tips-tricks-for-writing-shell-scripts-with-awesome-ux-19a525ae05ae

https://eng.localytics.com/exploring-cli-best-practices/

https://amir.rachum.com/blog/2017/07/28/python-entry-points/